YADA v0.1.0 by [me](https://github.com/Sxela) A minimal colab with support for init images, clip and lpips losses.

While we're all waiting for stable diffusion weights to go public, it's also fun to try running ye olde faithful latent diffusion model with the same settings and modes to get ready for day 1 :D

This is a fun project, don't take it seriously. Best, [Alex](https://twitter.com/devdef).

As usual, kudos to [Katherine Crowson](https://github.com/crowsonkb) for sticking all the fancy samplers in one [place](https://github.com/crowsonkb/k-diffusion)
and to [DamascusGit](https://github.com/DamascusGit) for pointers and the idea

In [2]:
!nvidia-smi

Sun Aug 21 19:41:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
try: 
  import k_diffusion as K
except: 
  !mkdir -p models/ldm/txt2img_f8_large/
  !wget -O models/ldm/txt2img_f8_large/model.ckpt https://ommer-lab.com/files/latent-diffusion/nitro/txt2img-f8-large/model.ckpt
  !git clone https://github.com/CompVis/stable-diffusion
  !pip install -e /content/stable-diffusion
  !pip install ipywidgets==7.7.1
  !pip install transformers==4.19.2

  !pip install omegaconf
  !pip install einops
  !pip install pytorch_lightning
  !pip install scikit-image
  !pip install opencv-python
  !pip install ai-tools
  !pip install cognitive-face
  !pip install zprint
  !pip install kornia

  !pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  !pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip

  %pip install lpips
  %pip install keras

  %cd /content/
  !git clone https://github.com/crowsonkb/k-diffusion/
  %cd k-diffusion
  !pip install -e .

  !pip uninstall pillow -y
  !pip install pillow

Restart session after installing, because some package here occasionally (or not) kills PIL

# k-diffusion

In [ ]:
%cd /content/k-diffusion
import argparse
import math,os,time

import accelerate
import torch
import torch.nn as nn
from tqdm.notebook import trange, tqdm

import k_diffusion as K
from pytorch_lightning import seed_everything

from omegaconf import OmegaConf
from ldm.util import instantiate_from_config

from torch import autocast
import numpy as np

from einops import rearrange
from torchvision.utils import make_grid
import PIL.Image
from torchvision import transforms

In [ ]:
class CFGDenoiser(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.inner_model = model

    def forward(self, x, sigma, uncond, cond, cond_scale):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        cond_in = torch.cat([uncond, cond])
        uncond, cond = self.inner_model(x_in, sigma_in, cond=cond_in).chunk(2)
        return uncond + (cond - uncond) * cond_scale
        
import gc 
def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    del pl_sd
    gc.collect()
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda().half()
    model.eval()
    return model

config = OmegaConf.load(f"/content/stable-diffusion/configs/latent-diffusion/txt2img-1p4B-eval.yaml")
model = load_model_from_config(config, f"/content/models/ldm/txt2img_f8_large/model.ckpt")

Settings

In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self


opt = AttrDict(
seed = 1134143,
batch_size = 1,
outdir = './outputs',
n_rows = 3,
n_iter = 2,
scale = 10,
C = 16, #4
f = 16, #16
H = 512 ,
W = 512,
fixed_code = False,
prompt = 'a gta 5 loading screen featuring snoop dogg',
ddim_steps = 100,
clip_guidance_scale = 200., #200
init_latent_scale = 0., #20
clip_type = 'ViT-L/14@336px',
prompt_clip = '',
init_vgg_scale = 0, #1000
clamp_grad = True,
clamp_max = 0.7,
strength = 1, #1 - ninit image, no skip steps
init_image = '/content/2fdaaee2-7a80-42cc-a524-59838ff40128_32.png',
dynamic_thresh = 2.
)

In [ ]:

if opt.prompt_clip == '':   opt.prompt_clip = opt.prompt

seed = opt.seed
batch_size = opt.batch_size
outdir = opt.outdir
n_rows = opt.n_rows
n_iter = opt.n_iter
scale = opt.scale
C = opt.C
f = opt.f
H = opt.H
W = opt.W
fixed_code = opt.fixed_code
prompt = opt.prompt
ddim_steps = opt.ddim_steps
clip_guidance_scale =opt.clip_guidance_scale
init_latent_scale = opt.init_latent_scale
clip_type = opt.clip_type
prompt_clip = opt.prompt_clip
init_vgg_scale = opt.init_vgg_scale


accelerator = accelerate.Accelerator()
device = accelerator.device
seed_everything(seed)
seeds = torch.randint(-2 ** 63, 2 ** 63 - 1, [accelerator.num_processes])
torch.manual_seed(seeds[accelerator.process_index].item())

os.makedirs(outdir, exist_ok=True)
outpath = outdir

n_rows =  n_rows if  n_rows > 0 else batch_size
sample_path = os.path.join(outpath, "samples")
os.makedirs(sample_path, exist_ok=True)
base_count = len(os.listdir(sample_path))
grid_count = len(os.listdir(outpath)) - 1

start_code = None
if fixed_code:
        start_code = torch.randn([batch_size, C, H // f, W // f], device=device)
precision_scope = autocast


assert prompt is not None
data = [batch_size * [prompt]]

assert 0. <= opt.strength <= 1., 'can only work with strength in [0.0, 1.0]'
t_enc = int(opt.strength * opt.ddim_steps)
print(f"target t_enc is {t_enc} steps")

INFO:pytorch_lightning.utilities.seed:Global seed set to 1134143


target t_enc is 100 steps


In [ ]:
import clip
from kornia import augmentation as KA
from torch.nn import functional as F
from resize_right import resize

def spherical_dist_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().pow(2).mul(2)


clip_model = clip.load(clip_type, device=device)[0].eval().requires_grad_(False)
clip_normalize = transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073),
                                          std=(0.26862954, 0.26130258, 0.27577711))
clip_size = (clip_model.visual.input_resolution, clip_model.visual.input_resolution)
aug = KA.RandomAffine(0, (1/14, 1/14), p=1, padding_mode='border')


In [ ]:
model_wrap = K.external.CompVisDenoiser(model)
sigma_min, sigma_max = model_wrap.sigmas[0].item(), model_wrap.sigmas[-1].item()

In [ ]:
from einops import rearrange, repeat

def make_cond_model_fn(model, cond_fn):
    def model_fn(x, sigma, **kwargs):
        with torch.enable_grad():
            x = x.detach().requires_grad_()
            denoised = model(x, sigma, **kwargs);# print(denoised.requires_grad)
            
            cond_grad = cond_fn(x, sigma, denoised=denoised, **kwargs).detach();# print(cond_grad.requires_grad)
            cond_denoised = denoised.detach() + cond_grad * K.utils.append_dims(sigma**2, x.ndim)
        return cond_denoised
    return model_fn


def make_static_thresh_model_fn(model, value=1.):
    def model_fn(x, sigma, **kwargs):
        return model(x, sigma, **kwargs).clamp(-value, value)
    return model_fn

def get_image_embed(x):
        if x.shape[2:4] != clip_size:
            x = resize(x, out_shape=clip_size, pad_mode='reflect')
        x = clip_normalize(x)
        x = clip_model.encode_image(x).float()
        return F.normalize(x)

def load_img(path):
    image = PIL.Image.open(path).convert("RGB")
    w, h = image.size
    print(f"loaded input image of size ({w}, {h}) from {path}")
    w, h = map(lambda x: x - x % 32, (w, h))  # resize to integer multiple of 32
    image = image.resize((opt.W, opt.H), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

init_image = opt.init_image
init_image = load_img(init_image).to(device)
init_image = repeat(init_image, '1 ... -> b ...', b=batch_size).half()





target_embed = F.normalize(clip_model.encode_text(clip.tokenize(prompt_clip, truncate=True).to(device)).float())
init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image)) 
x0 = init_latent

def cond_fn(x, t, denoised, **kwargs): #compare text clip embeds with denoised image embeds
        denoised = model_wrap_cfg.inner_model.inner_model.decode_first_stage(denoised);# print(denoised.requires_grad)
        image_embed = get_image_embed(aug(denoised.add(1).div(2)))
        loss = spherical_dist_loss(image_embed, target_embed).sum() * clip_guidance_scale
        grad = -torch.autograd.grad(loss, x)[0]
        return grad

def cond_fn(x, t, denoised, **kwargs): #compare init image latent with denoised latent 
        loss = spherical_dist_loss(denoised, init_latent).sum() * init_latent_scale
        print(loss)
        grad = -torch.autograd.grad(loss, x)[0]
        return grad

import lpips
lpips_model = lpips.LPIPS(net='vgg').to(device)

def cond_fn(x, t, denoised, **kwargs): 
        loss = 0. 
        grad = torch.zeros_like(x)
        if clip_guidance_scale>0:
          #compare text clip embeds with denoised image embeds
          denoised = model_wrap_cfg.inner_model.inner_model.differentiable_decode_first_stage(denoised);# print(denoised.requires_grad)
          image_embed = get_image_embed(aug(denoised.add(1).div(2)))
          loss = spherical_dist_loss(image_embed, target_embed).sum() * clip_guidance_scale
       
        if init_latent_scale>0:
          #compare init image latent with denoised latent 
          loss += spherical_dist_loss(denoised, init_latent).sum() * init_latent_scale
        
        if init_vgg_scale>0:
          #compare init image with denoised latent image via lpips
          denoised = model_wrap_cfg.inner_model.inner_model.differentiable_decode_first_stage(denoised)
          loss += lpips_model(denoised, init_image).sum() * init_vgg_scale
          
        if loss!=0. :
          grad = -torch.autograd.grad(loss, x)[0]
          # print(loss, grad.max())
          if opt.clamp_grad:
            magnitude = grad.square().mean().sqrt()
            return grad * magnitude.clamp(max=opt.clamp_max) / magnitude
        return grad

model_wrap_cfg = CFGDenoiser(model_wrap)
model_fn = make_cond_model_fn(model_wrap_cfg, cond_fn)
model_fn = make_static_thresh_model_fn(model_fn, opt.dynamic_thresh)

In [ ]:
seed_everything(seed)
with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                tic = time.time()
                all_samples = list()
                for n in trange(n_iter, desc="Sampling", disable =not accelerator.is_main_process):
                    for prompts in tqdm(data, desc="data", disable=not accelerator.is_main_process):
                        uc = None
                        if scale != 1.0:
                            uc = model.get_learned_conditioning(batch_size * [""])
                            
                        if isinstance(prompts, tuple):
                            prompts = list(prompts)
                        c = model.get_learned_conditioning(prompts)
                        
                        shape = [C, H // f, W // f]
                        sigmas = model_wrap.get_sigmas(ddim_steps)
                        extra_args = {'cond': c, 'uncond': uc, 'cond_scale': scale}
                        if opt.strength<1.0:
                          noise = torch.randn_like(x0) * sigmas[opt.ddim_steps - t_enc - 1] # for GPU draw
                          xi = x0 + noise
                          sigma_sched = sigmas[opt.ddim_steps - t_enc - 1:]
                          samples_ddim = K.sampling.sample_lms(model_fn, xi, sigma_sched, extra_args=extra_args, disable=not accelerator.is_main_process)
                        else: 
                          x = torch.randn([batch_size, *shape], device=device) * sigmas[0]
                          samples_ddim = K.sampling.sample_lms(model_fn, x, sigmas, extra_args=extra_args, disable=not accelerator.is_main_process)

                        x_samples_ddim = model.decode_first_stage(samples_ddim)
                        x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
                        x_samples_ddim = accelerator.gather(x_samples_ddim)        

                        if accelerator.is_main_process:
                            for x_sample in x_samples_ddim:
                                x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                                PIL.Image.fromarray(x_sample.astype(np.uint8)).save(
                                    os.path.join(sample_path, f"{base_count:05}.png"))
                                base_count += 1

                        if accelerator.is_main_process:
                            all_samples.append(x_samples_ddim)

                if accelerator.is_main_process:
                    # additionally, save as grid
                    grid = torch.stack(all_samples, 0)
                    grid = rearrange(grid, 'n b c h w -> (n b) c h w')
                    grid = make_grid(grid, nrow=n_rows)

                    # to image
                    grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
                    with open(os.path.join(outpath, f'grid-{grid_count:04}.txt'), "w") as text_file:
                        print(opt, file=text_file)
                    PIL.Image.fromarray(grid.astype(np.uint8)).save(os.path.join(outpath, f'grid-{grid_count:04}.png'))
                    grid_count += 1

                toc = time.time()

print(f"Your samples are ready and waiting for you here: \n{outpath} \n"
          f" \nEnjoy.")                
